# Part 1: Supplemental Background on Numpy

This notebook is a quick overview of additional functionality in Numpy. It is intended to supplement the videos and the other parts of this assignment. It does **not** contain any exercises that you need to submit. Understanding these additional NumPy functionalities will help you manipulate and analyze large datasets more efficiently.

In [ ]:
# Print Python and NumPy versions for reference

import sys
print(sys.version)

import numpy as np
print(np.__version__)

# Random numbers #

NumPy has a rich collection of pseudo-random number generators. "Pseudo-random" means that the numbers are generated using deterministic algorithms, which can produce sequences that appear random but are reproducible if the same initial conditions (or seed values) are used. This allows you to generate random numbers for simulations while still being able to replicate your results. See the documentation for [numpy.random](https://numpy.org/doc/stable/reference/random/index.html) for more details.

In [ ]:
from numpy.random import default_rng
rng = default_rng()

A = rng.integers(-10, 10, size=(4, 3)) # return random integers from -10 (inclusive) to 10 (exclusive)
print(A)

In [ ]:
# Display the generated matrix again for reference
A

In [ ]:
# Calculate the mean of each row in the original matrix A
np.mean(A.T, axis=0)

# Calculate the overall mean of all elements in A
np.mean(A.T)

# Aggregations or reductions #

Suppose you want to reduce the values of a Numpy array to a smaller number of values. Numpy provides a number of such functions that _aggregate_ values. Examples of aggregations include sums, min/max calculations, and averaging, among others.

In [ ]:
print("np.max =", np.max(A, axis=1),"; np.amax =", np.amax(A)) # np.max() and np.amax() are synonyms
print("np.min =",np.min(A),"; np.amin =", np.amin(A)) # same
print("np.sum =",np.sum(A))
print("np.mean =",np.mean(A))
print("np.std =",np.std(A))

The above examples aggregate over all values. However, you can also aggregate along a specific dimension using the optional `axis` parameter. 

The `axis` parameter specifies the dimension along which the aggregation is applied:
- `axis=0` aggregates over rows, producing results for each column.
- `axis=1` aggregates over columns, producing results for each row.

This can be confusing at first, this article provides an excellent explanation: [Numpy Axes, Explained](https://www.sharpsightlabs.com/blog/numpy-axes-explained/).

In [ ]:
print("Max in each column:", np.amax(A, axis=0)) # i.e., aggregate along axis 0, the rows, producing column maxes
print("Max in each row:", np.amax(A, axis=1)) # i.e., aggregate along axis 1, the columns, producing row maxes

# Universal functions

Universal functions apply a given function _elementwise_ to one or more Numpy objects.

For instance, `np.abs(A)` takes the absolute value of each element.

In [ ]:
print(A, "\n==>\n", np.abs(A)) # This outputs the original matrix A and its absolute value

Some universal functions accept multiple, compatible arguments. For instance, here, we compute the _elementwise maximum_ between two matrices, $A$ and $B$, producing a new matrix $C$ such that $c_{ij} = \max(a_{ij}, b_{ij})$.

> The matrices must have compatible shapes, which we will elaborate on below when we discuss Numpy's _broadcasting rule_.

In [ ]:
print(A) # recall A


In [ ]:
A.shape

In [ ]:
B = rng.integers(-10, 10, size=A.shape)
print(B)

In [ ]:
C = np.maximum(A, B) # elementwise comparison; for np.maximum to work, the two matrices must have the same shape or compatible dimensions.
print(C)

You can also build your own universal functions! Custom universal functions allow you to apply your own Python functions elementwise to NumPy arrays, enabling complex operations while retaining NumPy's performance benefits. For instance, suppose we want a way to compute, elementwise, $f(x) = e^{-x^2}$ and we have a scalar function that can do so:

In [12]:
def f(x):
    from math import exp
    return exp(-(x**2))

`np.frompyfunc(f, 1, 1)` creates a ufunc from the function `f`, where the first `1` is the number of input arguments and the second `1` is the number of output arguments. The following cell will create a new NumPy universal function, `f_np`. The function accepts one input (`x`) and returns a single output. See the documentation for [`np.frompyfunc()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.frompyfunc.html) for more details.


In [ ]:
f_np = np.frompyfunc(f, 1, 1) 

print(A, "\n=>\n", f_np(A))

# Broadcasting #

Sometimes, we want to combine operations on NumPy arrays that have different shapes but are _compatible_. **Broadcasting** allows operations on arrays of different shapes, as long as they follow specific rules for compatibility. This makes it possible to perform elementwise operations without manually reshaping arrays.

In the following example, we want to add `3` elementwise to every value in `A`.

In [ ]:
print(A)
print()
print(A + 3) # Adds 3 to every element in A, resulting in an array of the same shape as A.

Technically, `A` and `3` have different shapes: the former is a $4 \times 3$ matrix, while the latter is a scalar ($1 \times 1$). However, they are compatible because Numpy knows how to _extend_-- or **broadcast** -- the value 3 into an equivalent matrix object of the same shape in order to combine them.

To see a more sophisticated example, suppose each row `A[i, :]` represents the coordinates of a data point, and we want to compute the centroid of all the data points (or center-of-mass, if we imagine each point is a unit mass). The centroid is the average position of all data points in a space. It is found by taking the mean of the coordinates along each axis. This is equivalent to computing the mean coordinate for each column:

In [ ]:
A_col_means = np.mean(A, axis=0)  # Calculate the mean of each column.

print(A, "\n=>\n", A_col_means)

Suppose you want to shift the points so that each column's mean is zero. This means calculating the mean of each column and then subtracting that mean from each element in the corresponding column. Even though `A` and `A_col_means` don't have the same shape, NumPy will interpret `A - A_col_means` as precisely this operation, effectively extending or "replicating" `A_col_means` into each row of a matrix of the same shape as `A` to perform elementwise subtraction.

In [ ]:
A_centered_by_columns = A - A_col_means  # NumPy treats A_col_means as if it is replicated for each row, allowing elementwise subtraction with A to center the columns.
A_centered_by_columns

Suppose you instead want to mean-center the _rows_ instead of the columns. You could start by computing row means:

In [ ]:
A_row_means = np.mean(A, axis=1)  # Calculate the mean of each row.

print(A, "\n=>\n", A_row_means)

But the same operation will fail!

In [ ]:
A - A_row_means  # Fails!

The operation fails because `A` is a (4, 3) matrix while `A_row_means` is a (4,) array, making their shapes incompatible for direct subtraction.
 
**Broadcasting rule.** One way to fix this is to understand NumPy's convention for **[broadcasting](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#broadcasting)**. Broadcasting works if, starting from right to left, dimensions match or one of them is 1. This allows NumPy to "expand" smaller arrays to match larger ones.

In [ ]:
print(A.shape, A_col_means.shape)

These are compatible if, starting from _right_ to _left_, the dimensions match **or** one of the dimensions is 1. This convention of moving from right to left is referred to as matching the _trailing dimensions_. In this example, the rightmost dimensions of each object are both 3, so they match. Since `A_col_means` has no more dimensions, it can be replicated to match the remaining dimensions of `A`.

By contrast, consider the shapes of `A` and `A_row_means`:

In [ ]:
print(A.shape, A_row_means.shape)

In this case, per the broadcasting rule, the trailing dimensions of 3 and 4 do not match. Therefore, the broadcast rule fails. To make it work, we need to modify `A_row_means` to have a unit trailing dimension. Use NumPy's [`reshape()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) to convert `A_row_means` into a shape that has an explicit trailing dimension of size 1.

In [ ]:
A_row_means2 = np.reshape(A_row_means, (len(A_row_means), 1))
print(A_row_means2, "=>", A_row_means2.shape)

Reshape `A_row_means` to (4, 1) so that it matches the number of rows in `A` and can be broadcast across columns. Now the trailing dimension equals 1, so it can be matched against the trailing dimension of `A`. The next dimension is the same between the two objects, so NumPy knows it can replicate accordingly.

In [ ]:
print("A - A_row_means2\n\n", A, "\n-", A_row_means2) 
print("\n=>\n", A - A_row_means2)

**Thought exercise.** As a thought exercise, you might see if you can generalize and apply the broadcasting rule to a 3-way array.

# Matrix products #

Given two multidimensional arrays, there are several notions of "multiplying them" together. The two most common are the _elementwise_ (or _Hadamard_) product and _matrix multiplication_. The _Hadamard_ product is a fancy name for elementwise multiplication, where corresponding elements are multiplied together.

**Elementwise ("Hadamard") product.** Given two multidimensional array objects with the _same_ shape, the elementwise product is result of multiplying the corresponding elements.

For instance, recall `A` from above and suppose `B` has the same shape:

In [ ]:
print(A)

In [ ]:
B = rng.integers(-10, 10, size=A.shape)
print(B)

The elementwise product is an array `C` such that `C[i, j] = A[i, j] * B[i, j]`, which can be invoked via the `*` operator:

In [ ]:
A * B

Matrix multiplication: When multiplying two matrices, say $A$ of size $m \times k$ and $B$ of size $k \times n$, the result is a new matrix $C$ with size $m \times n$. Each element $c_{i,j}$ in matrix $C$ is calculated as the sum of the products of corresponding elements from the $i$-th row of $A$ and the $j$-th column of $B$:

$$
c_{i,j} = \sum_{s=0}^{k-1} a_{i, s} \cdot b_{s, j}
$$

For example, to compute the element `C[0, 0]`, we multiply each element in the first row of `A` by the corresponding element in the first column of `B`, then sum the results:

$$
c[0,0] = A[0,0] \cdot B[0,0] + A[0,1] \cdot B[1,0] + \ldots
$$

For a basic refresher on matrix multiplication, check out the [Khan Academy article on Multiplying Matrices](https://www.khanacademy.org/math/precalculus/x9e81a4f98389efdf:matrices/x9e81a4f98389efdf:multiplying-matrices-by-matrices/a/multiplying-matrices).

In [ ]:
print(f"A ==\n{A}\n")
B = rng.integers(-10, 10, size=(A.shape[1], 5))
print(f"B ==\n{B}")

The matrix product or matrix multiplication can be carried out using two possible constructs: `dot` and `@` perform matrix multiplication, with `@` being the more modern, preferred syntax.

In [ ]:
C = A.dot(B)
print(C)

In [ ]:
C = A @ B
print(C)

**Fin!** That marks the end of this notebook. If you want to learn more, checkout the second edition of [Python for Data Analysis](http://shop.oreilly.com/product/0636920050896.do) (released in October 2017).